In [44]:
import pandas as pd
import geopandas as gp
#import matplotlib.pyplot as plt
import os


In [45]:
bay_county_names = ["Alameda", "ContraCosta", "Sonoma", "Solano", "SanMateo", "SantaClara", "SanFrancisco", "Marin","Napa"] 
analysis_years = ["2008", "2009","2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017"]
slump_years = ["2008", "2009","2010", "2011", "2012"]
recovery_years = ["2013", "2014", "2015", "2016", "2017"]

In [46]:
crosswalk = pd.read_csv("data/downloaded_raw/geocorr2000.csv")
crosswalk.head()

,county,tract,FIPS state,placefp,State Postal Code,cntyname,Place Name,"Total Hsg. Units, 2000 census",tract to placefp allocation factor
0,6001,4001.0,6,53000,CA,Alameda CA,Oakland city,1177,1
1,6001,4002.0,6,53000,CA,Alameda CA,Oakland city,873,1
2,6001,4003.0,6,53000,CA,Alameda CA,Oakland city,2567,1
3,6001,4004.0,6,53000,CA,Alameda CA,Oakland city,1880,1
4,6001,4005.0,6,53000,CA,Alameda CA,Oakland city,1592,1


In [136]:
# hmda_geodata_09 = pd.read_csv("/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/data/geo_data/all_bay_concat_2009.csv").drop(["Unnamed: 0", "Unnamed: 0.1", "Unnamed: 0.1.1"], axis=1)
# hmda_geodata_09.head()

In [47]:
for year in analysis_years:
    file = pd.read_csv("/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/data/county_geo_data/all_bay_concat_"+year+".csv")
    merged = file.merge(crosswalk, how= 'left', left_on="Tract", right_on="tract")
    merged["Place Name"].replace(to_replace=' ', value='unincorporated non CDP', inplace=True)
    merged.to_csv("data/place_parsed/allbay_place_geolocated_"+str(year)+"_.csv")
    #these will be dataframes with place assigned to them, I will want to concatenate them with the years in the
    #defined range and then groupby average, so for now I should export them as individual years


In [49]:
path = "/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/data/place_parsed/"
for year in analysis_years:
    slump_csvs = []
    recovery_csvs = []
    slump_file_names = [f for f in os.listdir(path) 
                    if f[-3:] == "csv"
                    and f.split('_')[3] in slump_years]
    for file in slump_file_names:   
        df = pd.read_csv(path+file)
        slump_csvs.append(df)
    slump = pd.concat(slump_csvs, axis=0)
    recovery_file_names = [f for f in os.listdir(path) 
                    if f[-3:] == "csv"
                    and f.split('_')[3] in recovery_years]
    for file in recovery_file_names:
        df = pd.read_csv(path + file)
        recovery_csvs.append(df)
    recovery = pd.concat(recovery_csvs, axis=0)




In [50]:
recovery_place_df = recovery.groupby("Place Name").agg('mean')[['% HI borrower, LI tract', '# HI borrower, LI tract',
       '% HI borrower, HI tract', '# HI borrower, HI tract',
       '% LI borrower, LI tract', '# LI borrower, LI tract',
       '%LI borrower, HI tract', '# LI borrower, HI tract', 'STATEFP00',
       'COUNTYFP00', 'FIPS state']]
slump_place_df = slump.groupby("Place Name").agg('mean')[['% HI borrower, LI tract', '# HI borrower, LI tract',
       '% HI borrower, HI tract', '# HI borrower, HI tract',
       '% LI borrower, LI tract', '# LI borrower, LI tract',
       '%LI borrower, HI tract', '# LI borrower, HI tract', 'STATEFP00',
       'COUNTYFP00', 'FIPS state']]
slump_place_df["Period"] = "slump 2008-2012"
recovery_place_df["Period"] = "recovery 2013-2017"

In [51]:
recovery_place_df

,"% HI borrower, LI tract","# HI borrower, LI tract","% HI borrower, HI tract","# HI borrower, HI tract","% LI borrower, LI tract","# LI borrower, LI tract","%LI borrower, HI tract","# LI borrower, HI tract",STATEFP00,COUNTYFP00,FIPS state,Period
Place Name,,,,,,,,,,,,
Alameda city,0.257523,7.946667,0.679890,43.613333,0.022477,0.640000,0.040110,2.400000,6.0,1.000000,6.0,recovery 2013-2017
Alamo CDP,0.000000,0.000000,0.988195,100.800000,0.000000,0.000000,0.011805,1.080000,6.0,13.000000,6.0,recovery 2013-2017
Albany city,0.038462,0.038462,0.928727,35.461538,0.000000,0.000000,0.032812,2.076923,6.0,1.000000,6.0,recovery 2013-2017
Alum Rock CDP,0.776373,50.720000,0.000000,0.000000,0.223627,14.160000,0.000000,0.000000,6.0,85.000000,6.0,recovery 2013-2017
Angwin CDP,0.000000,0.000000,0.936462,54.800000,0.000000,0.000000,0.063538,3.800000,6.0,55.000000,6.0,recovery 2013-2017
Antioch city,0.357402,39.466667,0.220145,31.166667,0.342598,33.583333,0.079855,10.383333,6.0,13.000000,6.0,recovery 2013-2017
Ashland CDP,0.652431,39.466667,0.099735,5.133333,0.214236,12.933333,0.033599,1.633333,6.0,1.000000,6.0,recovery 2013-2017
Atherton town,0.000000,0.000000,0.984470,44.000000,0.000000,0.000000,0.015530,0.533333,6.0,81.000000,6.0,recovery 2013-2017
Bay Point CDP,0.301211,25.466667,0.262502,67.100000,0.365455,29.866667,0.070832,10.566667,6.0,13.000000,6.0,recovery 2013-2017


In [ ]:
recovery_place_df.to_csv(path+ "recovery_place_concat.csv")
slump_place_df.to_csv(path+"slump_place_concat.csv")

In [6]:
#need to geocode the recovery and slump aggregated place files using place census shapefiles
places = gp.read_file("/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/data/raw_shapefiles/pl06_d00_shp/pl06_d00.shp")
places["Merge Name"] = places["NAME"] + " " +places["LSAD_TRANS"]
places

,AREA,PERIMETER,PL06_D00_,PL06_D00_I,STATE,PLC,PLACEFP,NAME,LSAD,LSAD_TRANS,geometry,Merge Name
0,2.045401e-04,0.073558,2,1,06,0785,19584,Dorris,58,city,"POLYGON ((-121.914307 41.971627, -121.913242 4...",Dorris city
1,1.077178e-04,0.052906,3,2,06,2950,80686,Tulelake,58,city,"POLYGON ((-121.481851 41.953458, -121.480855 4...",Tulelake city
2,3.214861e-04,0.089923,4,3,06,1290,34694,Hornbrook,57,CDP,"POLYGON ((-122.573989 41.919534, -122.567163 4...",Hornbrook CDP
3,1.191514e-04,0.051211,5,4,06,0640,17022,Crescent City,58,city,"POLYGON ((-124.154424 41.858125, -124.142128 4...",Crescent City city
4,4.038516e-05,0.030405,6,5,06,1667,44812,Macdoel,57,CDP,"POLYGON ((-122.00627 41.830193, -122.004741 41...",Macdoel CDP
5,2.005628e-04,0.060585,7,6,06,1865,49768,Mount Hebron,57,CDP,"POLYGON ((-122.015291 41.79307, -122.003981 41...",Mount Hebron CDP
6,5.168609e-04,0.173153,8,7,06,0647,17030,Crescent City North,57,CDP,"POLYGON ((-124.248984 41.776697, -124.242691 4...",Crescent City North CDP
7,1.654652e-03,0.225863,9,8,06,0247,06150,Bertsch-Oceanview,57,CDP,"POLYGON ((-124.155962 41.773872, -124.154618 4...",Bertsch-Oceanview CDP
8,2.801575e-03,0.467153,10,9,06,3180,86944,Yreka,58,city,"POLYGON ((-122.586001 41.7681, -122.589451 41....",Yreka city
9,4.286751e-04,0.132903,11,10,06,0640,17022,Crescent City,58,city,"POLYGON ((-124.203694 41.769003, -124.19939 41...",Crescent City city


In [252]:
print(len(places.dropna(subset=["Merge Name"])))
print(len(places.dropna(subset=["Merge Name"]).drop_duplicates(subset=["Merge Name"])))


# unique_places = places.dropna(subset=["Merge Name"])["Merge Name"].unique()
unique_places = set([])
duplicate_places = set([])
all_places = places.dropna(subset=["Merge Name"])["Merge Name"].values
for place in all_places:
    if place not in unique_places:
        unique_places.add(place)
    else:
        duplicate_places.add(place)

# duplicate_places


test = places.dropna(subset=["Merge Name"])[places["Merge Name"] == 'Adelanto city']
test.to_csv("test.csv")
# print(len(places.dropna(subset=["Merge Name"])))
# print(len(recovery_place_df))

# # merged_recovery = recovery_place_df.merge(places.dropna(subset=["Merge Name"]), how='outer', left_on='Place', right_on='Merge Name')
# merged_recovery = recovery_place_df.merge(places.dropna(subset=["Merge Name"]), how='left', left_on='Place', right_on='Merge Name')

# print(len(merged_recovery))

# merged_recovery





1229
1081


/Users/ameliabaum/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:455: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super(GeoDataFrame, self).__getitem__(key)


In [24]:
recovery_csv = pd.read_csv("/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/data/place_parsed/recovery_place_concat.csv")
slump_csv = pd.read_csv("/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/data/place_parsed/slump_place_concat.csv")

In [26]:
places.head()

,AREA,PERIMETER,PL06_D00_,PL06_D00_I,STATE,PLC,PLACEFP,NAME,LSAD,LSAD_TRANS,geometry,Merge Name
0,0.000205,0.073558,2,1,06,0785,19584,Dorris,58,city,"POLYGON ((-121.914307 41.971627, -121.913242 4...",Dorris city
1,0.000108,0.052906,3,2,06,2950,80686,Tulelake,58,city,"POLYGON ((-121.481851 41.953458, -121.480855 4...",Tulelake city
2,0.000321,0.089923,4,3,06,1290,34694,Hornbrook,57,CDP,"POLYGON ((-122.573989 41.919534, -122.567163 4...",Hornbrook CDP
3,0.000119,0.051211,5,4,06,0640,17022,Crescent City,58,city,"POLYGON ((-124.154424 41.858125, -124.142128 4...",Crescent City city
4,0.000040,0.030405,6,5,06,1667,44812,Macdoel,57,CDP,"POLYGON ((-122.00627 41.830193, -122.004741 41...",Macdoel CDP


In [41]:
slump_geo = slump_csv.merge(places, how='left', left_on='Place Name', right_on='Merge Name').dropna()

recovery_geo = recovery_csv.merge(places, how='left', left_on='Place Name', right_on='Merge Name').dropna()


In [42]:
slump_geo_csv = slump_geo.to_csv('/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/data/place_geo/slump_geo_parsed.csv')
recovery_geo_csv = recovery_geo.to_csv('/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/data/place_geo/recoveru_geo_parsed.csv')